In [39]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import SimpleITK  as sitk
from glob import glob

In [ ]:
mask_path = './test_img'
points_path = './full_points.csv'

file_list = glob(os.path.join(mask_path,'*_13.jpg'))
print(len(file_list))

for file_name in file_list:
    file = file_name.split('/')[-1]
    name = file[:-7]
    
    ori_img = "./img/"+name+".jpg"
    
    ori_img = cv2.imread(ori_img,0)
    img = cv2.imread(file_name,0)
    mask = cv2.bitwise_not(img)
    
    h=mask.shape[0]
    w=mask.shape[1]
    print(mask.shape)
    new_points = centerpoint(points_path,file_name)
    
    
    try :
        crops = cropping_ver(new_points,mask)
        ori_crops = cropping_ver(new_points,ori_img)
    
        i=1
        for crop,ori_crop in zip(crops,ori_crops):
#             fast = cv2.fastNlMeansDenoising(crop, None, 20,7,25)
            
            Gau = cv2.GaussianBlur(crop, (0,0),3)
            result = drawROI(Gau)
            
            img_cal = result /255
            
            if round(img_cal.sum() / (img_cal.shape[0]*img_cal.shape[1])*100,2) > 27 and round(img_cal.sum() / (img_cal.shape[0]*img_cal.shape[1])*100,2) < 50:
                cv2.imwrite(os.path.join('./result2',name+'_'+str(i)+'.jpg'),result)
                fig = plt.figure()
                ax1 = fig.add_subplot(1, 3, 1)
                ax2 = fig.add_subplot(1, 3, 2)
                ax3 = fig.add_subplot(1, 3, 3)
                ax1.imshow(ori_crop,cmap='gray')
                ax2.imshow(crop,cmap='gray')
                ax3.imshow(result,cmap='gray')
                plt.show()
#             i+=1
#         for image in img_crops:
#             plt.imshow(image,'gray')
#         plt.show()
#             break
    except :
        print('crop error')
        
    

In [15]:
img = cv2.imread("./result2/20091122_1247534_ser001img00006_1.jpg",0)
img = img /255
print(np.histogram(img))
round(img.sum() / (img.shape[0]*img.shape[1])*100,2)

(array([136335,      0,      0,      0,      0,      0,      0,      0,
            0,  20465]), array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]))


13.05

In [2]:
mask_path = './test_img'
points_path = './full_points.csv'

In [4]:
file_list = glob(os.path.join(mask_path,'*_13.jpg'))
print(len(file_list))

184


In [ ]:
# file_list = file_list[:10]

for file_name in file_list:
    file = file_name.split('/')[-1]
    name = file[:-7]

    
    img = cv2.imread(file_name,0)
    mask = cv2.bitwise_not(img)
    
    h=mask.shape[0]
    w=mask.shape[1]
    print(mask.shape)
    new_points = centerpoint(points_path,file_name)
    crops = cropping_ver(new_points,mask)
    
    try :
        i=1
        for crop in crops:
#             fast = cv2.fastNlMeansDenoising(crop, None, 20,7,25)
            
            Gau = cv2.GaussianBlur(crop, (0,0),3)
            result = drawROI(Gau)
            
            plt.imshow(result,cmap='gray')
            plt.show()
            
            cv2.imwrite(os.path.join('./result',name+'_'+str(i)+'.jpg'),result)
            i+=1
#         for image in img_crops:
#             plt.imshow(image,'gray')
#         plt.show()
    except :
        print('crop error')
        
    

In [2]:
def centerpoint(points_path,file_name):
    file_name = file_name.split('/')[-1]
    name = file_name[:-7]+'.jpg'
    print(name)
    arr = pd.read_csv(points_path).to_numpy()
    keys = np.array(arr[...,2])
    keys_unique = np.unique(keys)
    new_points=[]
    for key_unique in keys_unique:
        vals= arr[np.where(arr[...,2]==key_unique)]
#     print(vals)
        for val in vals:
            if val[2]==name:
#             print(val)
                new_points.append(val)
    new_points = np.array(new_points)            
    print(new_points)
    return new_points

In [3]:
def cropping(new_points,mask):
    crops = []
    if len(new_points)<8:
        for new_point in new_points:
            center_x = new_point[3]
            center_y = new_point[4]
        #     rect = cv2.rectangle(mask, (center_x-200,center_y-150),(center_x+200,center_y+150),(255,255,255),-1)
            crops.append(mask[center_y-180:center_y+180, center_x-210:center_x+210])

    else :
        for new_point in new_points:
            center_x = new_point[3]
            center_y = new_point[4]
        #     rect = cv2.rectangle(mask, (center_x-200,center_y-150),(center_x+200,center_y+150),(255,255,255),-1)
            crops.append(mask[center_y-110:center_y+110, center_x-140:center_x+140])
    return crops

In [3]:
def cropping_ver(new_points,mask):
    y_list = []
    crops=[]
#     mask = mask.copy()
    for new_point in new_points:
        center_x = new_point[3]
        center_y = new_point[4]

        y_list.append(new_point[4])
    y_list = np.array(y_list)

    y_max = y_list.max()
    y_min = y_list.min()
    
    diff_ls=[]
    for i in range(len(y_list)-1):
    #     if i == len(y_list):
    #         break
        diff = y_list[i]-y_list[i+1]

        diff_ls.append(abs(diff))
        diff_np = np.array(diff_ls)

        if diff_np.min():


            y_diff = diff_np.min()

    y_border= y_diff
    for new_point in new_points:
        center_x = new_point[3]
        center_y = new_point[4]
        cropping = mask[center_y-y_border:center_y+y_border,center_x-200:center_x+200]
        crops.append(cropping)
    return crops


In [81]:
def cropping(new_points,mask):
    rect_list = []
    crops=[]
    crop_list = []
    mask = mask.copy()
    border = 5
    for new_point in new_points:
        center_x = new_point[3]
        center_y = new_point[4]
        rect = cv2.rectangle(mask, (center_x-110,center_y-80),(center_x+110,center_y+80),(255,255,255),-1)
        coords = center_x-110, center_y-80, center_x+110,center_y+80
        crops.append(mask[coords[1]:coords[3], coords[0]:coords[2]])
        print('1:',coords)
        
        rect_list.append(rect)
    
    
    
    for i in rect_list:
    #     img = i
        msk_error = i

        _, is_single = find_contours(msk_error)
        new_coords = coords
        print('3:',new_coords)
        cropping = mask[new_coords[1]:new_coords[3], new_coords[0]:new_coords[2]]
        crop_list.append(cropping)
        plt.imshow(cropping,'gray')
        ## Single Contour Case
        dilate = msk_error

        ## Multi Contour Case
        kernel_size = 35
        while not is_single:
            kernel = np.ones((kernel_size, kernel_size), np.uint8)
            dilate = cv2.dilate(msk_error, kernel, iterations = 3)
            new_coords = coords[0]-border,coords[1]-border,coords[2]+border,coords[3]+border
            print('2:',new_coords)
            cropping = mask[new_coords[1]:new_coords[3], new_coords[0]:new_coords[2]]
          
            
            _, is_single = find_contours(dilate)
            border += 5
            kernel_size += 2
            
            crop_list.append(cropping)
    plt.show()
    return crop_list

def find_contours(org_img):
    contours, _= cv2.findContours(org_img.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours, len(contours) == 1

In [4]:
def drawROI(img):
    ret, img = cv2.threshold(img, 70, 255, cv2.THRESH_BINARY)
    kernel = np.ones((3,3),np.uint8)
    img = cv2.morphologyEx(img,cv2.MORPH_CLOSE,kernel)
    # Find contours
    contours, _ = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    contours.sort(key=lambda x: len(x), reverse=True)
   
    # make mask
    if len(contours)>2:
       
        mask = np.zeros_like(img)
        mask = cv2.drawContours(mask, [contours[0]], 0, 255, -1)
        mask = cv2.drawContours(mask, [contours[1]], 0, 255, -1)

        # find two extreme points
        c = contours[0]
        fleft = tuple(c[c[:, :, 0].argmin()][0])
        fright = tuple(c[c[:, :, 0].argmax()][0])

        c = contours[1]
        sleft = tuple(c[c[:, :, 0].argmin()][0])
        sright = tuple(c[c[:, :, 0].argmax()][0])

        # connect
        mask = cv2.line(mask, fleft, sleft, 255, 5)
        mask = cv2.line(mask, fright, sright, 255, 5)

        # draw whole ROI
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        contours.sort(key=lambda x: len(x), reverse=True)

        mask = cv2.drawContours(mask, [contours[0]], 0, 255, -1)
    else :
        mask = np.zeros_like(img)
    return mask 

In [101]:
mask_path = './test_img/20140225_7809283_ser001img00002_63.jpg'
mask = cv2.imread(mask_path,0)

In [ ]:
new_points = centerpoint(points_path,mask_path)
crops = cropping(new_points,mask)
for crop in crops:
        Gau = cv2.GaussianBlur(crop, (0,0),1)
        result = drawROI(Gau)
        plt.imshow(result,'gray')
        plt.show()